In [17]:
from golden import trigger_v2
import datetime
import sqlalchemy
from sqlalchemy.orm import sessionmaker
from utilities import Market
import pandas as pd

In [2]:
a = trigger_v2('600844', datetime.date(2023, 1, 1), datetime.date(2024, 12, 3))

In [3]:
a

[{'ticker': '600844', 'signal': datetime.date(2023, 11, 24)},
 {'ticker': '600844', 'signal': datetime.date(2024, 5, 28)},
 {'ticker': '600844', 'signal': datetime.date(2024, 8, 9)},
 {'ticker': '600844', 'signal': datetime.date(2024, 12, 3)}]

In [ ]:
engine = sqlalchemy.create_engine('sqlite:///stockwatcher.db', echo=False, max_overflow=20, pool_size=20)
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
def golden_backtest(ticker_signal, windowsize:int):
    """
        信号出现次日收盘价持有windowsize，计算区间涨跌幅
    """
    for i in ticker_signal:
        _ticker = i['ticker']
        _signal = i['signal']
        _end = _signal + datetime.timedelta(days=windowsize)
        
        # 获取行情数据
        _price = pd.read_sql(session.query(Market).filter(Market.ticker == _ticker).statement, session.bind)
       # 过滤时间
        _price['tradeDate'] = pd.to_datetime(_price['tradeDate'])
        _price['tradeDate'] = _price['tradeDate'].dt.date
        _price1 = _price[(_price['tradeDate'] > _signal) & (_price['tradeDate'] <= _end)].copy()
        
        # 计算持有收益率
        

In [4]:
"""
    草稿
"""
import akshare as ak
import pandas as pd
import math

In [2]:
a = ak.stock_zh_a_hist(symbol="600719", period='daily', start_date="20241125",
                                        end_date="20241209",
                                        adjust='hfq', timeout=30)

In [1]:
from golden import amplitude_convergence

a = amplitude_convergence("600793", start_date="2023-01-01", end_date="2024-12-06", increase_range=8)

In [8]:
a

[        id   tradeDate  ticker   open  close   high    low       amount  \
 0  6836800  2023-12-12  600793  21.32  23.45  23.45  21.07  120085245.0   
 1  6836801  2023-12-13  600793  23.51  24.43  25.80  22.76  298991196.0   
 2  6836802  2023-12-14  600793  23.94  23.23  24.36  23.07  214181015.0   
 3  6836803  2023-12-15  600793  23.18  23.23  23.76  22.69  168258763.0   
 
    amplitude  perf_percent  perf_amount           createtime  \
 0      11.16          9.99         2.13  2024-11-30 21:00:49   
 1      12.96          4.18         0.98  2024-11-30 21:00:49   
 2       5.28         -4.91        -1.20  2024-11-30 21:00:49   
 3       4.61          0.00         0.00  2024-11-30 21:00:49   
 
             updatetime    volumn  turnover  std_conver  
 0  2024-11-30 21:00:49   95718.0      5.41    0.000000  
 1  2024-11-30 21:00:49  225214.0     12.73    0.064819  
 2  2024-11-30 21:00:49  166518.0      9.41    0.053227  
 3  2024-11-30 21:00:49  132271.0      7.48    0.046536  ,


In [1]:
from golden import amplitude_convergence, get_stocks
from utilities import MyThread
import math

num_threads = 20

def conver_pool(codelist, start_date, end_date):
    _results = []
    for i in codelist:
        try:
            _a = amplitude_convergence(i, start_date, end_date)
            if len(_a) > 0:
                _results.append(_a)
            else:
                continue
        except Exception as e:
            print(f"{i}, 错误码{e}")
            continue
    return _results


stock = get_stocks()
results = []
f = locals()
for thread in range(num_threads):
    delta = math.ceil(stock.shape[0] / num_threads)
    _stocklist = stock['ticker'].to_list()[thread * delta: (thread + 1) * delta]
    f['n_' + str(thread)] = MyThread(conver_pool, args=(_stocklist, "2024-01-01", "2024-12-11",))

for t in range(num_threads):
    f['n_' + str(t)].start()

for t in range(num_threads):
    f['n_' + str(t)].join()

# 获取结果
for t in range(num_threads):
    _result = f['n_' + str(t)].get_result()
    results.extend(_result)

In [2]:
len(results)

204

np.float64(1.4699999999999989)

In [4]:
import pandas as pd
yp = []
for i in results:
    _code = list(i.keys())[0]
    for j in i[_code]:
        _start = list(j.keys())[0]
        _signal = list(j[_start].keys())[0]
        _aveDist = j[_start][_signal]
        _d = pd.DataFrame(data={
            'ticker': [_code], 'trigger_date': [_start], 'signal_date': [_signal],
            'ave_distance': [_aveDist]
        })
        yp.append(_d)

In [5]:
yp1 = pd.concat(yp)

In [6]:
yp1.reset_index(drop=True, inplace=True)

In [7]:
yp1

,ticker,trigger_date,signal_date,ave_distance
0,000005,2024-01-26,2024-02-05,0.451667
1,000005,2024-02-21,2024-02-28,0.488000
2,000005,2024-02-22,2024-02-29,0.488000
3,000005,2024-02-23,2024-03-04,0.430000
4,000005,2024-02-26,2024-03-05,0.430000
...,...,...,...,...
666,601096,2024-04-18,2024-04-25,0.168000
667,601096,2024-09-19,2024-09-26,0.314000
668,301526,2024-02-06,2024-02-21,0.308000
669,301526,2024-09-30,2024-10-14,0.394000


In [ ]:
"""
    收敛要加上
"""